This notebook implements a rule-based decision fusion system that combines predictions from two models: DTW and LSTM, to classify hand trajectory sequences. The aim is to use the strengths of both models by switching between them based on certain heuristics.

At the beginning of the hand movement, DTW is used to make the first prediction as DTW typically performs well early in the gesture.

Then, once the sequence length crosses a certain threshold (more than 50 frames), the system checks if the LSTM prediction is stable (repeated same label for the last few predictions).

If LSTM predictions stabilize, the model switches to use LSTM as the final decision maker.

In [1]:
import cv2
import mediapipe as mp
import pyrealsense2 as rs
import numpy as np
import time
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
import joblib
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean
from collections import defaultdict, deque, Counter

In [11]:
# ─────────────── Load Models and Encoders ───────────────
lstm_model = load_model("lstm_index_finger_model_new.keras")
label_encoder = joblib.load("label_encoder_new.pkl")
templates_X = np.load("X_dtw_new.npy")
templates_y = np.load("y_dtw_new.npy")

# DTW setup
def setup_dtw_templates(X, y):
    ref = defaultdict(list)
    for i, label in enumerate(y):
        ref[label].append(X[i])
    return {label: seqs[0] for label, seqs in ref.items()}

templates = setup_dtw_templates(templates_X, templates_y)

def classify_with_dtw(sequence):
    best_label, best_distance = None, float('inf')
    for label, ref_seq in templates.items():
        dist, _ = fastdtw(sequence, ref_seq, dist=euclidean)
        if dist < best_distance:
            best_distance = dist
            best_label = label
    return best_label

# ─────────────── Rule-Based Fusion ───────────────
def use_lstm(trajectory, threshold=5.0):
    """Return True if movement is stable based on low variance."""
    traj_np = np.array(trajectory)
    if len(traj_np) < 10:
        return False
    last_part = traj_np[-10:]
    var = np.var(last_part, axis=0)  # variance in x, y, z
    return np.all(var < threshold)

def predict_lstm(sequence, max_len=50):
    padded = pad_sequences([sequence], maxlen=max_len, padding='post', truncating='post')
    probs = lstm_model.predict(padded, verbose=0)[0]
    return np.argmax(probs)

# ─────────────── Camera and Hand Setup ───────────────
MAX_LEN = 50
trajectory = []
recent_predictions = deque(maxlen=10)
stable_label = None

pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
pipeline.start(config)
align = rs.align(rs.stream.color)

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

try:
    while True:
        frames = pipeline.wait_for_frames()
        aligned = align.process(frames)
        depth_frame = aligned.get_depth_frame()
        color_frame = aligned.get_color_frame()

        if not depth_frame or not color_frame:
            continue

        color_image = np.asanyarray(color_frame.get_data())
        frame_rgb = cv2.cvtColor(color_image, cv2.COLOR_BGR2RGB)
        results = hands.process(frame_rgb)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(color_image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                h, w, _ = color_image.shape
                lm = hand_landmarks.landmark[8]
                cx, cy = int(lm.x * w), int(lm.y * h)
                cx, cy = np.clip(cx, 0, w - 1), np.clip(cy, 0, h - 1)
                z = depth_frame.get_distance(cx, cy) * 1000  # mm
                trajectory.append([cx, cy, z])

                if len(trajectory) > MAX_LEN:
                    trajectory = trajectory[-MAX_LEN:]

                if len(trajectory) >= MAX_LEN:
                    if use_lstm(trajectory):
                        pred_idx = predict_lstm(trajectory)
                    else:
                        pred_idx = classify_with_dtw(trajectory)

                    label = label_encoder.inverse_transform([pred_idx])[0]
                    recent_predictions.append(label)
                    most_common = Counter(recent_predictions).most_common(1)[0]

                    if most_common[1] >= 6:
                        stable_label = most_common[0]

                    if stable_label:
                        cv2.putText(color_image, f"Predicted: {stable_label}", (10, 60),
                                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

                cv2.circle(color_image, (cx, cy), 8, (0, 255, 0), -1)
                cv2.putText(color_image, f"{round(z)} mm", (cx + 10, cy - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 1)

        cv2.imshow("Real-Time Prediction", color_image)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

except KeyboardInterrupt:
    print("Interrupted")

finally:
    print("Exiting")
    pipeline.stop()
    cv2.destroyAllWindows()


Exiting
